#                                          Open Food Facts API - Product Weights

Importing Libraries

In [1]:
import requests
import pandas as pd 

Data Call

## Vegetables

In [5]:
%%time

url = "https://world.openfoodfacts.org/cgi/search.pl"
query = "vegetable"
page_size = 100  # Adjusted to 100 for a faster response (maximum is 200)
max_pages = 50  # Adjusted to 50 for testing purposes

product_names = []
weights = []
packaging_details = []
countries = []  # New list to store the country information

for page_number in range(1, max_pages + 1):
    params = {
        "search_terms": query,
        "json": 1,
        "page_size": page_size,
        "page": page_number,
        "tagtype_0": "categories",
        "tag_contains_0": "contains",
        "tag_0": "vegetables",
        "tagtype_1": "additives",
        "tag_contains_1": "without"
    }

    try:
        response = requests.get(url, params=params)
        response.raise_for_status()  # Raise an exception for 4xx or 5xx status codes
        data = response.json()
        products = data.get("products", [])

        if len(products) == 0:
            print(f"No more data found after page {page_number}. Exiting.")
            break

        for product in products:
            # Check if the product has weight information and its language is English
            if "quantity" in product and product.get("lang", "unknown") == "en":
                name = product.get("product_name", "Unknown Vegetable").strip()
                # Skip if the vegetable name is unknown
                if name == "Unknown Vegetable":
                    continue
                weight = product.get("quantity", "Unknown Weight").strip()
                packaging = product.get("packaging", "Unknown Packaging").strip()
                country = product.get("countries", "Unknown Country").strip()  # Extract country information

                # Append data to lists
                product_names.append(name)
                weights.append(weight)
                packaging_details.append(packaging)
                countries.append(country)  # Add country information to the list

    except requests.exceptions.RequestException as e:
        print(f"Error occurred while processing page {page_number}: {e}")
        break
    except Exception as e:
        print(f"Unexpected error occurred while processing page {page_number}: {e}")
        break

# Create a pandas DataFrame
data_dict = {
    'Product Name': product_names,
    'Weight': weights,
    'Packaging Details': packaging_details,
    'Country': countries  # Add the 'Country' column to the data_dict dictionary
}

CPU times: total: 2.39 s
Wall time: 1min 35s


In [6]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)
Veg = pd.DataFrame(data_dict)

Veg

,Product Name,Weight,Packaging Details,Country
0,Heinz Beanz In a rich tomato sauce,415 g,"Металл,en:Can,en:Canned,en:Food can","Франция,Португалия,Россия,Испания,Швеция,Великобритания"
1,Röstzwiebeln,100g,"Plastic,Pot,de:Stück,fr:Pot plastique","Canada,France,Germany,Switzerland"
2,Tomacouli,3 x 200 g,"Plastique,Brique,Carton,Etui en carton",France
3,Tomatenmark,130g,"Metal, Container, Dry, Tube","Australien, Frankreich, Italien, Niederlande, Spanien, Schweiz, Germany"
4,Passierte Tomaten,500g,"Tetra Pak, Pappe, Karton, da:C/PAP, 84","Deutschland,Rumänien"
5,Cebolla frita,150 g,"Plastic, PP - Polypropylene, Green Dot",España
6,Lentilles cuites dans très peu de jus,265 g,Conserve,France
7,Petit Croquants,185g,"Glass,Jar,Bocal en verre,Capsule en métal,","France,Allemagne,"
8,Tajine légume grillés,375 g,fr:R FR LE TRI +FACILE Å BAC DE TRI,France
9,Fritada pisto,380 g,Brick,Spain
